# Part 3: Serving

In this part you use MLRun's **serving runtime** to deploy your trained models from the previous stage, a `Voting Ensemble` using **max vote** logic.
You also use MLRun's **Feature store** to receive the latest tag of the online **Feature Vector** you defined in the previous stage.

By the end of this tutorial you’ll learn how to:
- Define a model class to load your models, run preprocessing, and predict on the data
- Define a Voting Ensemble function on top of your models
- Test the serving function locally using your `mock server`
- Deploy the function to the cluster and test it live

## Environment setup

First, make sure SciKit-Learn is installed in the correct version:

In [1]:
!pip install -U scikit-learn

Restart your kernel post installing.
Since your work is done in this project scope, you should define the project itself for all your MLRun work in this notebook.

In [2]:
project_name = "fraud-demo"

In [3]:
import mlrun

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name, context="./", user_project=True)

## Define model class
- Load models
- Predict from the feature store online service via the `source` key

In [4]:
# mlrun: start-code

In [5]:
import numpy as np
from cloudpickle import load
from mlrun.serving.v2_serving import V2ModelServer


class ClassifierModel(V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model(".pkl")
        self.model = load(open(model_file, "rb"))

    def predict(self, body: dict) -> list:
        """Generate model predictions from sample"""
        print(f"Input -> {body['inputs']}")
        feats = np.asarray(body["inputs"])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [6]:
# mlrun: end-code

## Define a serving function

MLRun serving can produce managed real-time serverless pipelines from various tasks, including MLRun models or standard model files.
The pipelines use the Nuclio real-time serverless engine, which can be deployed anywhere.
[Nuclio](https://nuclio.io/) is a high-performance open-source serverless framework that's focused on data, I/O, and compute-intensive workloads.

The **EnrichmentVotingEnsemble** and the **EnrichmentModelRouter** router classes auto-enrich the request with data from the feature store.
The router input accepts a list of inference requests (each request can be a dict or list of incoming features/keys). It enriches the request with data from the specified feature vector (`feature_vector_uri`).

In many cases the features can have null values (None, NaN, Inf, ..). The `Enrichment` routers can substitute the null 
value with fixed or statistical value per feature. This is done through the `impute_policy` parameter, which accepts the 
impute policy per feature (where `*` is used to specify the default). The value can be a fixed number for constants or 
`$mean`, `$max`, `$min`, `$std`, `$count` for statistical values, to substitute the value with the equivalent feature stats (taken from the feature store).  

The following code achieves:

- Gather ClassifierModel code from this notebook
- Define `EnrichmentVotingEnsemble` - Max-Vote based ensemble with feature enrichment and imputing
- Add the previously trained models to the ensemble

In [7]:
# Create the serving function from your code above
serving_fn = mlrun.code_to_function(
    "transaction-fraud", kind="serving", image="mlrun/mlrun"
).apply(mlrun.auto_mount())

serving_fn.set_topology(
    "router",
    "mlrun.serving.routers.EnrichmentVotingEnsemble",
    name="VotingEnsemble",
    feature_vector_uri="transactions-fraud-short",
    impute_policy={"*": "$mean"},
)

model_names = [
    "transaction_fraud_rf",
    "transaction_fraud_xgboost",
    "transaction_fraud_adaboost",
]

for i, name in enumerate(model_names, start=1):
    serving_fn.add_model(
        name,
        class_name="ClassifierModel",
        model_path=project.get_artifact_uri(f"{name}#{i}:latest"),
    )

# Plot the ensemble configuration
serving_fn.spec.graph.plot()

## Test the server locally

Before deploying the serving function, you can test it in the current notebook and check the model output.

In [8]:
# Create a mock server from the serving function
local_server = serving_fn.to_mock_server()

In [9]:
# Choose an id for your test
sample_id = "C1000148617"

model_inference_path = "/v2/models/infer"

# Send your sample ID for prediction
local_server.test(path=model_inference_path, body={"inputs": [[sample_id]]})

# notice the input vector is printed 3 times (once per child model) and is enriched with data from the feature store

### Accessing the real-time feature vector directly

You can also directly query the feature store values using the `get_online_feature_service` method. This method is used internally in the EnrichmentVotingEnsemble router class.

In [10]:
import mlrun.feature_store as fstore

# Create the online feature service
svc = fstore.get_online_feature_service(
    "transactions-fraud-short:latest", impute_policy={"*": "$mean"}
)

# Get sample feature vector
sample_fv = svc.get([{"source": sample_id}])
sample_fv

## Deploying the function on the Kubernetes cluster

You can now deploy the function. Once deployed, you get a function with http trigger that can be called from other locations.

Model activities can be tracked into a real-time stream and time-series DB. The monitoring data
is used to create real-time dashboards, detect drift, and analyze performance.<br>
To monitor a deployed model, apply `set_tracking()`.


In [11]:
import os

# Enable model monitoring
serving_fn.set_tracking()
project.set_model_monitoring_credentials(os.getenv("V3IO_ACCESS_KEY"))

# Deploy the serving function
serving_fn.deploy()

## Test the server

You can test the serving function and examine the model output.

In [12]:
# Choose an id for your test
sample_id = "C1000148617"

model_inference_path = "/v2/models/infer"

# Send your sample ID for prediction
serving_fn.invoke(path=model_inference_path, body={"inputs": [[sample_id]]})

You can also directly query the feature store values, which are used in the enrichment.

### Simulate incoming data

In [13]:
# Load the dataset
data = mlrun.get_dataitem(
    "https://s3.wasabisys.com/iguazio/data/fraud-demo-mlrun-fs-docs/data.csv"
).as_df()

# use only first 10k
data = data.sort_values(by="source", axis=0)[:10000]

# keys
sample_ids = data["source"].to_list()

In [14]:
from random import choice, uniform
from time import sleep

# Sending random requests
for _ in range(10):
    data_point = choice(sample_ids)
    try:
        resp = serving_fn.invoke(
            path=model_inference_path, body={"inputs": [[data_point]]}
        )
        print(resp)
        sleep(uniform(0.2, 1.7))
    except OSError:
        pass

## Done!

You've completed Part 3 of the deploying the serving function.
Proceed to [Part 4](04-pipeline.html) to learn how to automate ML Pipeline.

